In [1]:
import sys
import json
sys.path.append("/Users/chenry/code/MetabolicModelGapfilling/lib/")
from MetabolicModelGapfilling.core.gapfillingmodule import GapfillingModule
from IPython.display import IFrame

config = {
    "save_report_to_kbase":0,
    "template_file":"/Users/chenry/code/MetabolicModelGapfilling/data/Template.html",
    "scratch":"/Users/chenry/temp/",
    "workspace-url":"https://kbase.us/services/ws",
    "save_output_to_kbase":0
}
gfm = GapfillingModule(config)
html = gfm.run_gapfill_metabolic_model({
    "workspace":"patrikd:narrative_1605639637696",
    "fbamodel_id":"77537/357/1",
    "fbamodel_output_id":"GapfillTestModel.gftest",
    "media_ids" : ["77537/270/1"],
    "nogrow_media_ids" : [],
    "fbamodel_workspace" : "patrikd:narrative_1605639637696",
    "media_workspace" : "patrikd:narrative_1605639637696",
    "target_reaction" : "bio1",
    "source_fbamodel_id" : None,
    "source_fbamodel_workspace" : "patrikd:narrative_1605639637696",
    "feature_ko_list" : [],
    "reaction_ko_list" : [],
    "blacklist" : [],
    "media_supplement_list" : [],
    "minimum_target_flux" : 0.1,
    "max" : 1,
    "gapfilling_annotation_sources" : [],
    "consecutive_gapfill" : 1
},{
    "token":"YO6FO3VIEDX47PX5GWXQCZDMI62XFZWV"
})


cobrakbase 0.2.7


In [2]:
with open('view.html', 'w') as f:
    f.write(gfm.report_html)
IFrame(src="./view.html", width=1024, height=500)

In [1]:
import os
import sys
import cobra
import cobrakbase
import json
import csv
import logging
import cplex
import optlang
import re
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobrakbase.core.converters import KBaseFBAModelToCobraBuilder
from cobrakbase.Workspace.WorkspaceClient import Workspace as WorkspaceClient
from cobrakbase.core.kbase_object_factory import KBaseObjectFactory
from cobrakbase.core.fba_utilities import KBaseFBAUtilities
from cobra.core.dictlist import DictList
from cobra.core import Gene, Metabolite, Model, Reaction
from IPython.core.display import HTML


cobrakbase 0.2.7


In [4]:
params = {
    "workspace": 63927,
    "fbamodel_id" : 'TestModelUngapfilled',
    "fbamodel_workspace" : 63927,
	"media_id" : "ArgonneLBMedia",
	"media_workspace" : "KBaseMedia",
	"target_reaction" : "bio1",
	"source_fbamodel_id" : None,
	"source_fbamodel_workspace" : None,
	"feature_ko_list" : [],
	"reaction_ko_list" : [],
	"blacklist" : [],
	"custom_bound_list" : [],
	"media_supplement_list" : [],
	"minimum_target_flux" : 0.1,
    "max" : 1,
    "default_uptake" : 0,
    "default_excretion" : 100,
    "add_modelseed_reactions" : 1,
    "use_modelseed_peak_hits": 1,
    "gapfilling_annotation_sources" : []
}
kbase_api = cobrakbase.KBaseAPI()
media = kbase_api.get_from_ws(params["media_id"],params["media_workspace"])
kbmodel = kbase_api.get_object(params["fbamodel_id"],params["fbamodel_workspace"])
model = kbase_api.get_from_ws(params["fbamodel_id"],params["fbamodel_workspace"])
model.solver = 'cplex'
utilities = KBaseFBAUtilities(model,kbmodel,kbase_api,media = media,
                 default_uptake = 100,default_excretion = 100,blacklist = [])
#Saving the list of original reactions
#original_reactions = {}
#for reaction in model.reactions:
#    original_reactions[reaction.id] = 1
#Computing reaction scores from ontology
reaction_genes = utilities.compute_reaction_scores(1)#Optional 
#Extending model for gapfilling
utilities.default_uptake = 0
penalties = utilities.build_model_extended_for_gapfilling(1,[],[],2,reaction_genes)
#Setting objective function
utilities.set_objective_from_target_reaction(params["target_reaction"],params["max"])
#Setting minimal biomass production
utilities.convert_objective_to_constraint(params["minimum_target_flux"],None)
#Setting minimal reaction objective
utilities.create_minimal_reaction_objective(penalties,default_penalty = 0)
#Printing LP file
with open('Gapfill.lp', 'w') as out:
    out.write(str(model.solver))
#Minimizing gapfilled reactions
gapfilling_solution = model.optimize()
output = utilities.compute_gapfilled_solution(penalties)
print(len(output["reversed"]))
print(len(output["new"]))
#print(len(json.dumps(output,indent=2))
#Ensuring minimality by adding binary variables
flux_values = utilities.binary_check_gapfilling_solution(penalties,1)
#Reporting solution
output = utilities.compute_gapfilled_solution(penalties,flux_values)
print(len(output["reversed"]))
print(len(output["new"]))
#print(json.dumps(output,indent=2))

2
49
2
49
